In [52]:
import numpy as np
import matplotlib.pyplot as plt

In [53]:
#Using code from mzp5-wykresy
class NumericalMethod2d():
    def __init__(self,updates = None):
        self.updates = updates
        
    def add_step(self,update):
        if self.updates is None:
            self.updates = update
        else:
            self.updates = np.vstack((self.updates,update))
        
    @property
    def update_count(self):
        return self.updates.shape[0]-1
    
    def __call__(self,f1,f2,tn,y1n,y2n,h):
        k = np.zeros(shape=(self.update_count,2))
        for i in range(self.update_count):
            a = self.updates[i,0]
            b = self.updates[i,1:]
            k[i,0] = h * f1(tn+a*h,y1n+b.dot(k[:,0].T),y2n+b.dot(k[:,1].T))
            k[i,1] = h * f2(tn+a*h,y1n+b.dot(k[:,0].T),y2n+b.dot(k[:,1].T))
        c = self.updates[self.update_count,1:]
        return c.dot(k[:,0].T),c.dot(k[:,1].T)

In [106]:
def pendulum(method, start_x, start_y, start_dy, f1, f2, step = 0.1, saving = False):
    import matplotlib.animation as animation
    x = start_x
    y = start_y
    dy = start_dy
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-30,30)
    plt.xlim(-30,30)

    line, = ax.plot([], [], 'o-', lw=2)
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    
    def init():
        line.set_data([], [])
        time_text.set_text('')
        y = start_y
        x = start_x
        dy = start_dy
        return line, time_text
    
    def animate(t):
        global x
        global y
        global dy
        y_update, dy_update = method(f1=f1,f2=f2,tn=x,y1n=y,y2n=dy,h=step)
        y += y_update
        dy += dy_update
        line.set_data([0.0,-L*np.sin(y)],[0.0,-L*np.cos(y)])
        time_text.set_text(time_template % (t*step))
        return line, time_text
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if saving:
        ani.save('pendulum.mp4', fps=15)
    plt.show()
    x = start_x
    y = start_y
    dy = start_dy

In [55]:
euler = NumericalMethod2d(np.array([[1.,0.],[0.,1.]]))

In [56]:
heuns = NumericalMethod2d(
    np.array([[0.,0.,0.],
              [1.,1.,0.],
              [0.,.5,.5]]))

In [57]:
merson = NumericalMethod2d(
    np.array([[0.,0.,0.,0.,0.,0.],
              [1./3.,1./3.,0.,0.,0.,0.],
              [1./3.,1./6.,1./6.,0.,0.,0.],
              [1./2.,1./8.,0.,3./8.,0.,0.],
              [1.,1./2.,0.,-3./2.,2.,0.],
              [0.,1./6.,0.,0.,2./3.,1./6.]]))

## Zadanie 3

zagadnienie początkowe, parametry wahadła

In [98]:
Theta_0 = 3.
g=10.
L=15.
x=0.
y=Theta_0
dy=0.

animacja, zapisywanie do pliku przy fladze saving = True

In [99]:
pendulum(heuns,0,Theta_0,0,lambda t,y,v: v, lambda t,y,v:-g/L*np.sin(y), saving = False)

## Zadanie 4: Rownanie z uwzględnieniem oporów ruchu

In [108]:
Theta_0 = 3.
g=10.
L=15.
x=0.
y=Theta_0
dy=0.
c = .2
m = 0.4

In [ ]:
pendulum(heuns,
         0,
         Theta_0,
         0,
         lambda t,y,v: v,
         lambda t,y,v:-c*v/(m*L)-g/L*np.sin(y), saving = True)

In [101]:
pendulum(euler,
         0,
         Theta_0,
         0,
         lambda t,y,v: v,
         lambda t,y,v:-c*v/(m*L)-g/L*np.sin(y))